### 📈 Curvature Optimization

Once the spline is built:
- You must **replace the assumed boundary conditions** (like natural spline: second derivative = 0 at endpoints).
- Your goal is to **optimize** the spline such that the **total curvature is minimized**.
- This will result in a **smoother path**—which is desirable for autonomous vehicle trajectory planning.


### 🔍 What does Optimization mean here?
There are 3 parts to an optimization problem:
- Objective (or Cost) Function :
This is the function you want to minimize or maximize.
- Parameters (or Decision Variables):
These are the variables you are allowed to change in order to achieve the best value of the objective.
- Constraints:
These are the rules or limits that the parameters and/or the solution must satisfy.
In general, optimization means adjusting parameters to minimize or maximize an objective — like reducing cost, increasing speed, or in our case: minimizing curvature.

Here, you are given 4 fixed waypoints. Your goal is to interpolate these points using cubic splines, but with a twist:

refer to the first 3 videos of the following playlist for a better understanding:
[Video: Introduction to Curve Optimization](https://youtube.com/playlist?list=PLLK3oSbvdxFdF67yVxF_1FQO9SbBY3yTL&si=EMnkQ3Gb_EleDISu)

---
## 🎯 PROBLEM STATEMENT
You are given 4 discrete waypoints from the previous section. Your task is to interpolate these points using **cubic spline interpolation**, but with a focus on **optimizing the shape of the curve**.

Essentially, you are required to **minimize the total curvature** of the resulting spline by **varying the second derivative values at the boundaries**. This should be done **without using inbuilt spline functions** from libraries such as `scipy`. The goal is to find the **smoothest possible path** through the points, which would represent the **ideal trajectory** for a vehicle to follow. You may choose suitable bounds for the second derivative values and iterate through different combinations of boundary conditions to determine the configuration that leads to the **minimum total curvature**.

---

## Curvature Formula

The **curvature** $\kappa(t)$ of a 2D parametric curve $(x(t), y(t))$ is given by:

$$
\kappa(t) = \frac{ |x'(t)y''(t) - y'(t)x''(t)| }{ \left( (x'(t))^2 + (y'(t))^2 \right)^{3/2} }
$$

Where:
- $x'(t)$ and $y'(t)$ are the **first derivatives** with respect to the parameter $t$
- $x''(t)$ and $y''(t)$ are the **second derivatives**

---

## 📌 Deliverables

1. Your own implementation of cubic spline interpolation.
2. Plots of:
   - Original points
   - Interpolated spline without optimization
   - Interpolated spline with optimization
---

### 🌟 Optional Assignement

You may notice that directly finding the smoothest curve through waypoints seems inefficient and computationally expensive. Fortunately, mathematicians have developed robust optimization techniques that are readily available through libraries like `scipy.optimize`.

In this task, try enhancing the **cubic interpolation curve** you obtained earlier by finding an **optimal trajectory through the waypoints** — one that minimizes the curvature at every interpolated point. To do this, you can treat the **second derivatives at the curve’s boundaries** as variables to be optimized. Use `scipy` functions to perform this optimization and compare the resulting smooth curve to the original interpolation.

However, you may not notice an appreciable reduction in sharpness and a smoother, more natural path. What else can be done then for the **optimal raceline** for our car?

---

**Good luck!**  Let the paths be smooth and the math be satisfying!

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
x=[]
y=[]
x=np.array(list(eval(input("enter x values"))))
y=np.array(list(eval(input("enter y values"))))

SyntaxError: invalid syntax (<string>, line 0)

In [ ]:
matrix= np.zeros((12,12))
variables=' [a1,b1,c1,d1,a2,b2,c2,d2,a3,b3,c3,d3]'
print(variables+"\n", matrix)

In [ ]:
for i in range(4):
    matrix[0][i]=x[0]**(3-i)
    matrix[1][i]=x[1]**(3-i)
    matrix[2][i+4]=x[1]**(3-i)
    matrix[3][i+4]=x[2]**(3-i)
    matrix[4][i+8]=x[2]**(3-i)
    matrix[5][i+8]=x[3]**(3-i)
    
    if 2-i>=0:
        matrix[6][i]=(3-i)*x[1]**(2-i)
        matrix[6][i+4]=-(3-i)*x[1]**(2-i)
        
        matrix[8][i+4]=(3-i)*x[2]**(2-i)
        matrix[8][i+8]=-(3-i)*x[2]**(2-i)

    if 1-i>=0:
        matrix[7][i]=(3-i)*(2-i)*x[1]**(1-i)
        matrix[7][i+4]=-(3-i)*(2-i)*x[1]**(1-i)

        matrix[9][i+4]=(3-i)*(2-i)*x[2]**(1-i)
        matrix[9][i+8]=-(3-i)*(2-i)*x[2]**(1-i)
    
        matrix[10][i]=(3-i)*(2-i)*x[0]**(1-i)
        matrix[11][i+8]=(3-i)*(2-i)*x[3]**(1-i)

print (matrix)

In [ ]:
rhs=np.zeros(12)
rhs[0]=y[0]
rhs[5]=y[3]
for i in range(12):
    if i in range(2,5,2):
        rhs [i]=rhs[i-1]=y[i//2]
print(rhs)

In [ ]:
coeffs=np.linalg.solve(matrix,rhs)
print(coeffs)


def S1(x):
    s=0
    for i in range(0,4):
       s+=coeffs[i]*x**(3-i)
    return s 
def S2(x):
    s=0
    for i in range(4,8):
       s+=coeffs[i]*x**((3-i)%4)
    return s 
def S3(x):
    s=0
    for i in range(8,12):
       s+=coeffs[i]*x**((11-i)%4)
    return s

x1 = np.linspace(x[0], x[1], 100)
x2 = np.linspace(x[1],x[2],100)
x3 = np.linspace(x[2], x[3],100)

y1 = S1(x1)
y2 = S2(x2)
y3 = S3(x3)

plt.plot(x1, y1, label='S1(x)', color='red')
plt.plot(x2, y2, label='S2(x)', color='green')
plt.plot(x3, y3, label='S3(x)', color='blue')
plt.scatter(x, y, color='black', zorder=5, label='Data Points')
plt.title("Cubic Spline Interpolation")
plt.legend()
plt.grid (True)
plt.show